In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/default/1/adapter_model.safetensors
/kaggle/input/mistral/pytorch/default/1/MISTRAL.ipynb
/kaggle/input/mistral/pytorch/default/1/training_args.bin
/kaggle/input/mistral/pytorch/default/1/adapter_config.json
/kaggle/input/mistral/pytorch/default/1/README.md
/kaggle/input/mistral/pytorch/default/1/tokenizer.json
/kaggle/input/mistral/pytorch/default/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/default/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/default/1/tokenizer.model
/kaggle/input/dataset1/NewsSumm_perfect_clean.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from transformers import AutoTokenizer
file_to_check = "/kaggle/input/dataset1/NewsSumm_perfect_clean.csv"
df=pd.read_csv(file_to_check)
cluster_sizes = df.groupby('cluster_id').size()

valid_clusters = cluster_sizes[cluster_sizes >= 2].index

df_multi = df[df['cluster_id'].isin(valid_clusters)].reset_index(drop=True)

print("Filtered rows:", len(df_multi))
print("Filtered clusters:", df_multi['cluster_id'].nunique())
print("Avg docs per cluster:",
      df_multi.groupby('cluster_id').size().mean())
from sklearn.model_selection import train_test_split

clusters = df_multi['cluster_id'].unique()

train_clusters, temp_clusters = train_test_split(
    clusters, test_size=0.2, random_state=42
)

val_clusters, test_clusters = train_test_split(
    temp_clusters, test_size=0.5, random_state=42
)

train_df = df_multi[df_multi['cluster_id'].isin(train_clusters)]
val_df   = df_multi[df_multi['cluster_id'].isin(val_clusters)]
test_df  = df_multi[df_multi['cluster_id'].isin(test_clusters)]

print("Train clusters:", train_df['cluster_id'].nunique())
print("Val clusters:", val_df['cluster_id'].nunique())
print("Test clusters:", test_df['cluster_id'].nunique())
def build_cluster_samples(df):
    samples = []
    for cid, group in df.groupby("cluster_id"):
        docs = []

        # sort only if column exists
        if "published_date" in group.columns:
            try:
                group = group.sort_values("published_date")
            except Exception:
                pass

        for _, row in group.iterrows():
            docs.append(f"[DOC]\n{row['article_clean']}")

        samples.append({
            "cluster_id": cid,
            "source": "\n".join(docs),
            "summary": group.iloc[0]['summary_clean']
        })
    return samples
train_samples = build_cluster_samples(train_df)
val_samples   = build_cluster_samples(val_df)
test_samples  = build_cluster_samples(test_df)

print("Train samples:", len(train_samples))
print("Val samples:", len(val_samples))
print("Test samples:", len(test_samples))



Filtered rows: 4335
Filtered clusters: 2060
Avg docs per cluster: 2.104368932038835
Train clusters: 1648
Val clusters: 206
Test clusters: 206
Train samples: 1648
Val samples: 206
Test samples: 206


In [ ]:
!pip install bert_score

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/kaggle/input/mistral/pytorch/default/1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

model.eval()


`torch_dtype` is deprecated! Use `dtype` instead!
2026-02-04 19:52:54.464162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770234774.487700     222 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770234774.493669     222 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770234774.532010     222 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770234774.532028     222 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770234774.532033     222

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['alora_invocation_tokens', 'arrow_config', 'ensure_weight_tying', 'peft_version'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_featur

In [4]:
def generate_summaries_batch(texts, max_input_len=2048, max_new_tokens=120):

    prompts = [
        f"""Summarize the following news articles into a concise summary.

{text}

Summary:
""" for text in texts
    ]

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_input_len
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            eos_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [d.split("Summary:")[-1].strip() for d in decoded]


In [5]:
predictions = []
references = []

for sample in test_samples[:50]:
    predictions.append(generate_summary(sample["source"]))
    references.append(sample["summary"])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


5/50


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


10/50


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

40/50


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


45/50


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


50/50


In [9]:
!pip install -q evaluate rouge-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00


In [10]:
import evaluate

rouge = evaluate.load("rouge")

rouge_scores = rouge.compute(
    predictions=predictions,
    references=references
)

print("ROUGE RESULTS")
print(f"ROUGE-1 : {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2 : {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L : {rouge_scores['rougeL']:.4f}")


ROUGE RESULTS
ROUGE-1 : 0.3785
ROUGE-2 : 0.2179
ROUGE-L : 0.2823


In [11]:
!pip install -q bert-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
from bert_score import score

P, R, F1 = score(
    predictions,
    references,
    lang="en",
    rescale_with_baseline=True
)

print("BERTSCORE RESULTS")
print(f"Precision : {P.mean():.4f}")
print(f"Recall    : {R.mean():.4f}")
print(f"F1        : {F1.mean():.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTSCORE RESULTS
Precision : 0.1602
Recall    : 0.3909
F1        : 0.2708


In [13]:
for i in range(5):
    print(f"\n--- SAMPLE {i+1} ---")
    print("PREDICTED SUMMARY:")
    print(predictions[i])
    print("\nREFERENCE SUMMARY:")
    print(references[i])



--- SAMPLE 1 ---
PREDICTED SUMMARY:
Summarize the following news articles into a concise summary.

[DOC]
Brazilian leftist leader Luiz Inacio Lula da Silva narrowly defeated President Jair Bolsonaro in a runoff election, but the far right incumbent had not conceded defeat by Monday morning, raising concerns he might contest the result.Tens of thousands of jubilant supporters took to the streets of Sao Paulo to celebrate a stunning comeback for the 77-year-old former metalworker who, following his previous two-term 2003-2010 presidency, served prison time for corruption convictions that were later annulled.Bolsonaro is the first Brazilian incumbent to lose a presidential election and Lula has vowed to overturn his legacy, including pro-gun policies and weak protection of the Amazon rainforest.Pitching the contest as a battle for democracy after his rival made baseless claims the electoral system was open to fraud, Lula called the election a sign Brazilians "want more and not less democ